## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database_df.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,-46.6000,168.3333,64.80,66,100,14.05,NZ,moderate rain
1,1,Souillac,-20.5167,57.5167,77.38,83,40,6.91,MU,scattered clouds
2,2,Letnyaya Stavka,45.4306,43.4472,58.96,72,100,6.73,RU,overcast clouds
3,3,Vaini,-21.2000,-175.2000,77.16,94,100,9.22,TO,light intensity drizzle
4,4,Cape Town,-33.9258,18.4232,60.91,79,0,4.61,ZA,clear sky


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is hottest tempurature you desire on your trip?"))
min_temp= float(input("What is the coldest tempurature you desire on your trip?"))

What is hottest tempurature you desire on your trip?95
What is the coldest tempurature you desire on your trip?55


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
optimal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=  max_temp)&
                                    (city_data_df["Max Temp"] >= min_temp)]
optimal_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,-46.6000,168.3333,64.80,66,100,14.05,NZ,moderate rain
1,1,Souillac,-20.5167,57.5167,77.38,83,40,6.91,MU,scattered clouds
2,2,Letnyaya Stavka,45.4306,43.4472,58.96,72,100,6.73,RU,overcast clouds
3,3,Vaini,-21.2000,-175.2000,77.16,94,100,9.22,TO,light intensity drizzle
4,4,Cape Town,-33.9258,18.4232,60.91,79,0,4.61,ZA,clear sky


In [29]:
# 4a. Determine if there are any empty rows.
optimal_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             3
Current Description    0
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= optimal_cities_df.dropna()
clean_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,-46.6000,168.3333,64.80,66,100,14.05,NZ,moderate rain
1,1,Souillac,-20.5167,57.5167,77.38,83,40,6.91,MU,scattered clouds
2,2,Letnyaya Stavka,45.4306,43.4472,58.96,72,100,6.73,RU,overcast clouds
3,3,Vaini,-21.2000,-175.2000,77.16,94,100,9.22,TO,light intensity drizzle
4,4,Cape Town,-33.9258,18.4232,60.91,79,0,4.61,ZA,clear sky


In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,-46.6000,66,moderate rain,168.3333,64.80,
1,Souillac,-20.5167,83,scattered clouds,57.5167,77.38,
2,Letnyaya Stavka,45.4306,72,overcast clouds,43.4472,58.96,
3,Vaini,-21.2000,94,light intensity drizzle,-175.2000,77.16,
4,Cape Town,-33.9258,79,clear sky,18.4232,60.91,
5,Busselton,-33.6500,87,clear sky,115.3333,54.99,
7,Nanortalik,60.1432,84,overcast clouds,-45.2371,38.01,
8,Fortuna,40.5982,85,overcast clouds,-124.1573,56.66,
9,Dikson,73.5069,94,light snow,80.5464,16.43,
10,Mehamn,71.0357,86,light snow,27.8492,28.35,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params ["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found, skipping to next city")

Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping

Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping

Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping to next city
Hotel not found, skipping

In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.loc[(hotel_df["Hotel Name"] != '')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
43,Vardo,70.3705,65,light snow,31.1107,28.09,Village Lamera
86,Serenje,-13.2325,87,clear sky,30.2352,56.61,املاکی علی
113,Nanga Eboko,4.6833,77,light rain,12.3667,75.40,One Tierra - Konnakad
116,Gagarin,55.5529,67,few clouds,34.9954,34.02,Faros Hotel Ayia Napa
203,Lipin Bor,60.2622,83,broken clouds,37.9763,29.86,Bozan Koyu Muhtarligi


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vaction.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))